# Time Separated Model
This origin comming from https://www.kaggle.com/code/lblhandsome/optiver-robust-best-single-model/notebook

In [125]:
from pathlib import Path
import os
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling

# 📦 Importing machine learning libraries
import joblib  # For saving and loading models
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques

# 🤐 Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

max_lookback = np.nan  # Maximum lookback (not specified)

warnings.filterwarnings("ignore")

seed = 2023
num_folds = 5 # クロスバリデーションの分割数

# For kaggle environment
if os.environ.get("KAGGLE_DATA_PROXY_TOKEN") != None:
    BASE_OUTPUT_PATH = Path(f'/kaggle/working')
    BASE_INPUT_PATH = Path(f'/kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')
    
    IS_LOCAL = False # If kaggle environment, set False
    IS_TRAIN = True # If kaggle environment, set True
    IS_INFER = True # If kaggle environment, set True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    USE_OPTUNA = True # Use optuna or not

    if IS_LOCAL:
        SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
        REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')

    stopping_rounds = 3 # early_stopping用コールバック関数
    num_boost_round = 30 # 計算回数
    DEVICE = 'gpu' # cpu or gpu

    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE,
        'verbosity': -1
    }


    """
    lgb_params = {
        'task': 'train',                   # 学習
        'objective': 'regression',                # 目的関数の種類。ここでは回帰タスクを指定
        'metric': 'rmse',                          # 評価指標
        'boosting_type': 'gbdt',                  # ブースティングタイプ。勾配ブースティング決定木
        "n_estimators": 32,                        # ブースティングに使用する木の数。多いほど性能が向上するが計算コストが増加
        "num_leaves": 64,                         # 木に存在する最大の葉の数。大きい値は精度を向上させるが過学習のリスクが増加
        "subsample": 0.8,                         # 各木のトレーニングに使用されるデータの割合。過学習を防ぐために一部のデータをサンプリング
        "colsample_bytree": 0.8,                  # 木を構築する際に使用される特徴の割合。特徴のサブセットを使用し過学習を防ぐ
        "learning_rate": 0.01,                 # 学習率。小さい値は堅牢なモデルを生成するが収束に時間がかかる
        'max_depth': 32,                           # 木の最大の深さ。深い木は複雑なモデルを作成するが過学習のリスクがある
        "device": DEVICE,                         # トレーニングに使用するデバイス（CPUまたはGPU）
        "verbosity": -1,                          # LightGBMのログ出力のレベル。-1はログを出力しないことを意味する
        "importance_type": "gain",                # 特徴重要度を計算する際の指標。"gain"は分割による平均情報利得
        'lambda_l1': 0.5,                         # L1正則化項の係数。過学習を防ぐためにモデルの複雑さにペナルティを課す
        'lambda_l2': 0.5,                         # L2正則化項の係数。同じく過学習を防ぐ
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
    }
    """

# For local environment
else:
    BASE_OUTPUT_PATH = Path(f'../output')
    BASE_INPUT_PATH = Path(f'../kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')

    IS_LOCAL = True
    IS_TRAIN = True
    IS_INFER = True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    USE_OPTUNA = True
    TARGET_STOCK_IDS = [0,1,2,3,4,5,6,7,8,9]

    # For training
    stopping_rounds = 1 # early_stopping用コールバック関数
    num_boost_round = 1 # 計算回数
    DEVICE = 'cpu' # cpu or gpu

    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE,
        'verbosity': -1
    }


print(f"BASE_OUTPUT_PATH: {BASE_OUTPUT_PATH}")
print(f"BASE_INPUT_PATH: {BASE_INPUT_PATH}")
print(f"TRAIN_FILE: {TRAIN_FILE}")
print(f"TEST_FILE: {TEST_FILE}")
print(f"IS_LOCAL: {IS_LOCAL}")
print(f"IS_TRAIN: {IS_TRAIN}")
print(f"IS_INFER: {IS_INFER}")
print(f"IS_USE_SAVED_MODEL: {IS_USE_SAVED_MODEL}")


BASE_OUTPUT_PATH: ../output
BASE_INPUT_PATH: ../kaggle/input/optiver-trading-at-the-close
TRAIN_FILE: ../kaggle/input/optiver-trading-at-the-close/train.csv
TEST_FILE: ../kaggle/input/optiver-trading-at-the-close/test.csv
IS_LOCAL: True
IS_TRAIN: True
IS_INFER: True
IS_USE_SAVED_MODEL: False


# Model Train Results

In [126]:
%%time 

from gc import collect;
from psutil import Process;
from os import system, getpid, walk;

# Defining global configurations and functions:-

    
def GetMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";


collect();
print(GetMemUsage())

RAM memory GB usage = 1.239
CPU times: user 179 ms, sys: 17.7 ms, total: 197 ms
Wall time: 196 ms


# Functions

In [127]:
%%time 

from typing import Sequence, Tuple
import pandas as pd

# for local execution
class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = [TEST_FILE, REVEALED_TARGETS_FILE, SAMPLE_SUBMISSION_FILE]
        self.group_id_column: str = 'time_id'
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 31.9 µs


In [128]:
def pd_display_max():
    pd.set_option('display.max_rows', None)  # 行の最大表示数を無制限に設定
    pd.set_option('display.max_columns', None)  # 列の最大表示数を無制限に設定
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

def pd_clear_display_max():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

In [129]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, name: str):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)


    print(f"Memory usage of {name} is {start_mem:.2f} MB")
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization is: {end_mem:.2f} MB")
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage

    start_mem = df.memory_usage().sum() / 1024**2
    return df

In [130]:
# 🏎️ Import Numba for just-in-time (JIT) compilation and parallel processing
from numba import njit, prange

# 📊 Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # 🚫 Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# 📈 Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [131]:
# 📊 Function to generate imbalance features
def imbalance_features(df):
    if DEVICE == 'gpu':
        import cudf
        df = cudf.from_pandas(df)
    
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1 features
    # Calculate various features using Pandas eval function
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("ask_price + bid_price")/2
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("imbalance_size-matched_size")/df.eval("matched_size+imbalance_size")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    # Create features for pairwise price imbalances
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
        
    # V2 features
    # Calculate additional features
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['match_balance'] = ( df['matched_size']  + (df['imbalance_buy_sell_flag'] * df['imbalance_size'])) / df['matched_size']
    
    # Calculate various statistical aggregation features
    
        
    # V3 features
    # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    if DEVICE == 'gpu':
        df = df.to_pandas()
    # Replace infinite values with 0
    return df.replace([np.inf, -np.inf], 0)

def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

# 📅 Function to generate time and stock-related features
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes

    # Map global features to the DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df


def cal_vix(df, group_key, target_col, period):
    return df.groupby(group_key)[target_col].transform(lambda x: np.log(x).diff().rolling(period).std())


def generate_historical_features(df, is_train):
    for col in ['wap', 'match_balance']:
        for window in [1, 2, 3, 10]:
            col_name = f"{col}_diff_{window}"
            df[col_name] = df.groupby("stock_id")[col].diff(window)
            #df[col_name] = df[col_name].fillna(0)  # NaNを0で置き換える
        for period in [5]:
            col_name = f"{col}_vix_{period}"
            df[col_name] = cal_vix(df, ['stock_id', 'date_id'], col, period)

    df = df.replace([np.inf, -np.inf], 0)
    return df

# 🚀 Function to generate all features by combining imbalance and other features
def generate_all_features(df, is_train):
    prev_cols = list(df.columns)

    # Generate imbalance features
    df = imbalance_features(df)
    df = numba_imb_features(df)
    df = generate_historical_features(df, is_train)
    df = other_features(df)
    
    generated_feature_name = list(set(df.columns) - set(prev_cols))
    gc.collect()  # Perform garbage collection to free up memory

    df = reduce_mem_usage(df, 'features')
    return df, generated_feature_name

def normarized_features(df):
    df['normalized_wap'] = (df['wap'] - global_population_wap['mean']) / global_population_wap['std']
    df['normalized_match_balance'] = (df['match_balance'] - global_population_mathch_balance['mean']) / global_population_mathch_balance['std']
    return df

# Generationg train dataset

In [132]:
def load_train_dataset():
    df = pd.read_csv(TRAIN_FILE)
    # 🧹 Remove rows with missing values in the "target" column
    df = df.dropna(subset=["target"])
    # 🔁 Reset the index of the DataFrame and apply the changes in place
    df.reset_index(drop=True, inplace=True)
    return df


In [133]:
%%time

# Check if the code is running in offline or online mode
df_train = load_train_dataset()

if IS_LOCAL:
    # In local mode, stock id TARGET_STOCK_ID is used for training
    df_train = df_train[df_train["stock_id"].isin(TARGET_STOCK_IDS)]

if IS_TRAIN:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }

    df_train, generated_feature_name = generate_all_features(df_train, True)

    #global_population_wap = df_train['wap'].describe()
    #global_population_mathch_balance = df_train['match_balance'].describe()

    # normarize
    # df_train = normarized_features(df_train)

    print("Build Online Train Feats Finished.")

collect()
print(GetMemUsage())

Memory usage of features is 258.35 MB
Memory usage after optimization is: 125.89 MB
Decreased by 51.27%
Build Online Train Feats Finished.
RAM memory GB usage = 1.824
CPU times: user 10.1 s, sys: 2.15 s, total: 12.2 s
Wall time: 13.2 s


In [144]:
# feature selection
"""
feature_name = [
    "wap_vix_5", "reference_price_shift_10", "matched_size_ret_10",
    "matched_size_shift_10", "match_balance_vix_5", "ask_price_bid_price_reference_price_imb2",
    "seconds_in_bucket", "match_balance_diff_10", "imbalance_size_ret_10",
    "ask_size_diff_10", "imbalance_size_shift_10", "reference_price",
    "ask_price_bid_price_imb", "reference_price_ret_10", "all_sizes_mean",
    "matched_size", "bid_size_diff_10", "volume", "reference_price_shift_3",
    "bid_size", "bid_price_wap_reference_price_imb2", "ask_size",
    "ask_size_diff_3", "reference_price_bid_price_imb", "reference_price_wap_imb",
    "all_prices_kurt", "matched_size_bid_size_ask_size_imb2", "bid_price",
    "wap_diff_10", "bid_size_diff_3", "all_prices_std", "bid_size_ask_size_imbalance_size_imb2",
    "all_prices_skew", "all_sizes_skew", "bid_price_wap_imb",
    "ask_price_diff_10", "ask_price_wap_imb", "imbalance_size_ret_3",
    "matched_size_shift_3", "reference_price_shift_1", "all_prices_mean",
    "imbalance_size", "matched_size_ret_3", "reference_price_shift_2",
    "ask_price", "ask_size_diff_2", "bid_size_diff_2", "price_pressure",
    "reference_price_ask_price_imb", "matched_size_shift_1", "bid_size_diff_1",
    "market_urgency", "wap_diff_3", "price_spread", "all_sizes_std",
    "matched_size_shift_2", "imbalance_size_shift_3", "bid_price_diff_10",
    "ask_size_diff_1", "far_price", "reference_price_ret_3", "match_balance",
    "wap_diff_2", "wap_diff_1", "matched_imbalance", "wap"
]"""
feature_name = generated_feature_name
feature_name

['all_sizes_std',
 'wap_diff_2',
 'wap_diff_3',
 'global_std_size',
 'all_sizes_kurt',
 'all_prices_std',
 'minute',
 'match_balance_diff_10',
 'global_median_price',
 'global_ptp_price',
 'bid_size_ask_size_imbalance_size_imb2',
 'all_prices_kurt',
 'all_prices_skew',
 'matched_size_bid_size_imbalance_size_imb2',
 'all_sizes_skew',
 'global_std_price',
 'all_sizes_mean',
 'ask_price_bid_price_reference_price_imb2',
 'bid_price_wap_reference_price_imb2',
 'all_prices_mean',
 'dow',
 'wap_diff_10',
 'match_balance_diff_1',
 'ask_price_wap_reference_price_imb2',
 'wap_diff_1',
 'match_balance_diff_2',
 'seconds',
 'match_balance_diff_3',
 'match_balance_vix_5',
 'ask_price_bid_price_wap_imb2',
 'matched_size_bid_size_ask_size_imb2',
 'global_ptp_size',
 'wap_vix_5',
 'matched_size_ask_size_imbalance_size_imb2',
 'global_median_size']

## global_stock_id_feats

In [135]:
df_global_stock_id_feats = pd.DataFrame(global_stock_id_feats)
df_global_stock_id_feats

,median_size,std_size,ptp_size,median_price,std_price,ptp_price
stock_id,,,,,,
0,42739.16,132986.920030,5898989.29,1.999695,0.003353,0.017414
1,25548.50,66444.908534,693898.57,1.999827,0.005588,0.029370
2,26228.10,75674.654248,1069837.58,2.000200,0.005333,0.051622
3,41667.00,93875.770520,1928848.21,1.999980,0.002903,0.018551
4,34014.58,80670.274550,1604065.54,1.999816,0.003717,0.017379
5,28287.94,108045.498152,2265456.70,2.000082,0.006301,0.072807
6,24720.61,109732.904460,949838.58,1.999907,0.004339,0.018155
7,29798.00,139523.985990,1383196.54,1.999987,0.005259,0.022833
8,57259.95,387634.914143,3047677.20,1.999989,0.004006,0.017607


## df_train_feats

In [136]:
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,match_balance_vix_5,dow,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,...,NaN,0,0,0,42739.160156,132986.921875,5.898990e+06,1.999695,0.003353,0.017414
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,...,NaN,0,0,0,25548.500000,66444.906250,6.938986e+05,1.999827,0.005588,0.029370
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,...,NaN,0,0,0,26228.099609,75674.656250,1.069838e+06,2.000200,0.005333,0.051622
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,...,NaN,0,0,0,41667.000000,93875.773438,1.928848e+06,1.999980,0.002903,0.018551
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,...,NaN,0,0,0,34014.578125,80670.273438,1.604066e+06,1.999816,0.003717,0.017379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237697,5,480,540,1.063551e+06,1,0.998403,11476997.00,0.999396,0.999396,0.998119,...,0.011378,0,0,9,28287.939453,108045.500000,2.265457e+06,2.000082,0.006301,0.072807
5237698,6,480,540,9.985981e+05,1,1.000118,13713463.00,1.000495,1.000469,0.999984,...,0.018601,0,0,9,24720.609375,109732.906250,9.498386e+05,1.999907,0.004339,0.018155
5237699,7,480,540,2.469864e+06,-1,0.996621,73054344.00,0.996304,0.996335,0.996573,...,0.013453,0,0,9,29798.000000,139523.984375,1.383196e+06,1.999987,0.005259,0.022833
5237700,8,480,540,8.815596e+05,-1,1.000927,84052704.00,1.000055,1.000602,1.000703,...,0.002727,0,0,9,57259.949219,387634.906250,3.047677e+06,1.999989,0.004006,0.017607


# Model Training

In [146]:
# 📦 Import necessary libraries
import numpy as np
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import gc
import os
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np
from dataclasses import dataclass
import optuna
import sys
import shutil

import optuna.integration.lightgbm as optuna_lgb
optuna.logging.set_verbosity(optuna.logging.ERROR)
import lightgbm as lgb

from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

@dataclass
class Model:
    model: lgb.Booster
    fold: int
    feature_importance: pd.DataFrame
    score: float
    best_iteration: int
    train_time: float = None
    weight: float = None
    mem_usage: float = None

def train_model(train_x, train_y, val_x, val_y, best_params=None):
    print("------ Model training start ------")
    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y, reference=trains)

    verbose_eval = 0
    if best_params is None:
        params = lgb_params
    else:
        params = best_params

    print("Use params:")
    print(params)

    model = lgb.train(
        params,
        trains,
        valid_sets=[valids, trains], # 検証データ
        valid_names=['Train', 'Valid'],    # データセット名前
        num_boost_round=num_boost_round,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval)
        ]
    )

    print("------ Model training end ------")
    return model

def cross_train(df, key, n_splits, feature_name, valid_name, best_params=None):
    """ For Cross Train

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """
    print("----------------------------------------")
    print(f"Cross Train key id {key}: start, shape: {df.shape}, n_splits: {n_splits}")
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    models = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"{key}: {fold} start")
        now_time = time.time()
        X_train, X_valid = df[feature_name].iloc[train_indices], df[feature_name].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")


        model = train_model(X_train, y_train, X_valid, y_valid, best_params)

        y_valid_pred = model.predict(X_valid)
        
        # For Debug
        #y_valid_df = pd.DataFrame(y_valid, columns=[valid_name])
        #y_valid_df["pred"] = y_valid_pred
        #y_valid_df.to_csv('filename.csv', index=False)

        score = mean_absolute_error(y_valid, y_valid_pred)
        train_time = time.time() - now_time
        mem_usage = sys.getsizeof(model) / (1024 * 1024) # MB
        m = Model(model, fold, model.feature_importance(), score, model.best_iteration, train_time, weight= 1 / n_splits, mem_usage=mem_usage)
        print(f"{key}: {fold} end, score: {score}, time: {m.train_time}, best_iteration: {m.best_iteration}, memory usage: {m.mem_usage}")
        
        models.append(m)
        
        del X_train, X_valid, y_train, y_valid
        gc.collect()

    print(f"Cross train {key} model len {len(models)}")
    print("----------------------------------------")
    return key, models

def optuna_tuning(df, n_splits, feature_name, valid_name):
    params = {
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'verbosity': -1
    }

    # TBD: spint dataframe by length
    train_x = df[feature_name]
    train_y = df[valid_name] 
    
    trains = optuna_lgb.Dataset(train_x, train_y)
    
    print("------- Optuna Tuning Start-------")
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    folds = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    verbose_eval = 0
    tuner = optuna_lgb.LightGBMTunerCV(
        params,
        trains,
        num_boost_round=num_boost_round,
        folds=folds,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval)
        ]
    )
    
    tuner.run()
    best_params = tuner.best_params
    print(" Params: ")
    for key, value in best_params.items():
        print(" {}: {}".format(key, value))
    print("------- Optuna Tuning End-------")
    return best_params

In [147]:
%%time

# Train
best_params = None
if USE_OPTUNA:
    best_params = optuna_tuning(df=df_train, n_splits=num_folds, feature_name=feature_name, valid_name="target")

key_models = df_train.groupby("seconds_in_bucket").apply(lambda x: cross_train(df=x, key=x.name, n_splits=num_folds, feature_name=feature_name, valid_name="target", best_params=best_params))
model_dict = {key: model for key, model in key_models}

if IS_USE_SAVED_MODEL:
    model_save_base_path = f"{BASE_OUTPUT_PATH}/model"
    if os.path.exists(model_save_base_path):
        print(f"{model_save_base_path} already exists, clean up it.")
        shutil.rmtree(model_save_base_path)
    os.makedirs(model_save_base_path)

    key_model_paths = []
    for key, models in key_models:
        model_save_path = f"{model_save_base_path}/{key}"
        os.makedirs(model_save_path)
        model_paths = []
        for model in models:
            model_save_fullpath = f"{model_save_path}/model_{key}_{model.fold}.txt"
            model.model.save_model(model_save_fullpath)
            model_paths.append(model_save_fullpath)
        key_model_paths.append((key, model_paths))

    model_dict_saved = {key: model_paths for key, model_paths in key_model_paths}
    print(model_dict_saved)

del df_train
collect()
print(GetMemUsage())

------- Optuna Tuning Start-------
num_boost_round: 1, stopping_rounds: 1, folds: 5


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.82183 + 0.093477


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.82467 + 0.0932847


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.8245 + 0.0935752


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.82467 + 0.093521


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.84657 + 0.0929911


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.82272 + 0.0928293


feature_fraction, val_score: 8.821828: 100%|##########| 7/7 [00:06<00:00,  1.02it/s]


Training until validation scores don't improve for 1 rounds
Did not meet early stopping. Best iteration is:
[1]	cv_agg's valid rmse: 8.82456 + 0.0931777


KeyboardInterrupt: 

In [139]:
# Show results
print("")
print(f"Total model len {(sum([len(models) for key, models in key_models]))}")
print(f"Total model mem usage {sum([sum([model.mem_usage for model in models]) for key, models in key_models]):2f} MB")
print(f"Model dict len {len(model_dict)}")


Total model len 275
Total model mem usage 0.014687 MB
Model dict len 55


In [140]:
# Check model quality
data = []

for key, i_models in model_dict.items():
    for model in i_models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

df_model = pd.DataFrame(data)
df_model.describe()

,key,fold,score,best_iteration,train_time
count,275.000000,275.000000,275.000000,275.0,275.000000
mean,270.000000,2.000000,6.077108,1.0,0.069578
std,159.034496,1.416792,1.030951,0.0,0.014693
min,0.000000,0.000000,4.565445,1.0,0.060244
25%,130.000000,1.000000,5.443119,1.0,0.063557
50%,270.000000,2.000000,5.831177,1.0,0.065558
75%,410.000000,3.000000,6.242689,1.0,0.069067
max,540.000000,4.000000,9.370074,1.0,0.238630


In [141]:
# Initialize an empty DataFrame for aggregated importances
aggregated_importance = pd.DataFrame(index=feature_name, columns=['importance'])

# Aggregate the importances from each model
for key, i_models in model_dict.items():
    for model in i_models:
        importance = pd.DataFrame({'feature': feature_name, 'importance': model.feature_importance})
        aggregated_importance = aggregated_importance.add(importance.set_index('feature'), fill_value=0)

aggregated_importance['importance'] /= len(df_model)

#pd_display_max()
# Sort the features by importance
aggregated_importance = aggregated_importance.sort_values(by='importance', ascending=False)
aggregated_importance

,importance
wap_vix_5,7.065455
wap_diff_1,6.425455
bid_price_wap_reference_price_imb2,6.345455
reference_price_shift_10,5.785455
wap_diff_2,5.778182
...,...
matched_size_shift_3,0.629091
price_spread,0.421818
matched_size_shift_1,0.287273
matched_size_shift_2,0.192727


In [142]:
# Clean up
del key_models
if IS_USE_SAVED_MODEL:
    print("Delete model_dict")
    del model_dict
collect()
print(GetMemUsage())

RAM memory GB usage = 1.827


# Infer

In [143]:
%%time

# 📉 Define a function to adjust prices based on volumes
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)  # 🧮 Calculate standard error based on volumes
    step = np.sum(prices) / np.sum(std_error)  # 🧮 Calculate the step size based on prices and standard error
    out = prices - std_error * step  # 💰 Adjust prices by subtracting the standardized step size
    return out

def model_infer(key, df_feat):
    def predictor(models):
        print(f"Predictor target models len {len(models)}")
        #print(f"Predictor Feat len {len(df_feat)}")
        predictions = np.zeros(len(df_feat))
        predictions = np.mean([model.predict(df_feat[feature_name]) for model in models], 0)
        return predictions
    
    if IS_USE_SAVED_MODEL:
        model_paths = model_dict_saved[key]
        models = [lgb.Booster(model_file=model_path) for model_path in model_paths]
        predictions = predictor(models)
        del models
    else:
        models = [m.model for m in model_dict[key]]
        predictions = predictor(models)
    collect()
    return predictions


y_min, y_max = -64, 64
predictions = []
cache = pd.DataFrame()
counter = 0

if IS_INFER:
    if IS_LOCAL:
        print("Infer Local")
        env = make_env()
    else:
        print("Infer Submission")
        import optiver2023
        env = optiver2023.make_env()
    iter_test = env.iter_test()

    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        print(f"------- counter {counter} start -------")

        # It faults due to test is iterator
        #seconds_in_bucket = test['seconds_in_bucket'][0]
        #print(f"prdict: {test['date_id'][0]}, {seconds_in_bucket}")
        
        # Generate cahce
        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        cache = reduce_mem_usage(cache, 'cache')
        if counter > 0:
            # 🔄 If not the first iteration, limit the cache to the last 21 rows for each stock
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        print(f"cache len {len(cache)}")

        # 📊 Generate features
        df_test, generated_feature_name = generate_all_features(cache, False)
        df_test = df_test[-len(test):].reset_index(drop=True)
        df_feat = df_test[feature_name]
        print(f"df_feat len {len(df_feat)}")

        # Get seconds_in_bucket and date_id
        seconds_in_bucket = df_test['seconds_in_bucket'][0]
        date_id = df_test['date_id'][0]
        print(f"Predict: {date_id}, {seconds_in_bucket}")
        
        # Predict
        predictions = model_infer(seconds_in_bucket, df_feat)

        # Adjust the predictions based on the order book imbalance
        zerosum_predictions = zero_sum(predictions, test['bid_size'] + test['ask_size'])
        clipped_predictions = np.clip(zerosum_predictions, y_min, y_max)
        clipped_predictions.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

        # Submit
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        
        # Clean up
        del df_feat, df_test,predictions, zerosum_predictions, clipped_predictions
        collect()
        counter += 1

collect()
print(GetMemUsage())

Infer Local
------- counter 0 start -------
Memory usage of cache is 0.02 MB
Memory usage after optimization is: 0.01 MB
Decreased by 55.96%
cache len 200
Memory usage of features is 0.11 MB
Memory usage after optimization is: 0.09 MB
Decreased by 16.59%
df_feat len 200
Predict: 478, 0
Predictor target models len 5
------- counter 1 start -------
Memory usage of cache is 0.18 MB
Memory usage after optimization is: 0.14 MB
Decreased by 22.59%
cache len 400
Memory usage of features is 0.22 MB
Memory usage after optimization is: 0.19 MB
Decreased by 16.60%
df_feat len 200
Predict: 478, 10
Predictor target models len 5
------- counter 2 start -------
Memory usage of cache is 0.26 MB
Memory usage after optimization is: 0.20 MB
Decreased by 22.60%
cache len 600
Memory usage of features is 0.33 MB
Memory usage after optimization is: 0.28 MB
Decreased by 16.60%
df_feat len 200
Predict: 478, 20
Predictor target models len 5
------- counter 3 start -------
Memory usage of cache is 0.35 MB
Memory